In [1]:
!pip install implicit==0.4.4

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
#from google.colab import files

#files.upload()

Saving retail_train.csv to retail_train (1).csv


In [3]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [6]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.head()

,item_id,sales_value
0,25671,20.94
1,26081,0.99
2,26093,1.59
3,26190,1.54
4,26355,1.98


In [7]:
popularity = data.groupby('item_id')['sales_value'].count().reset_index()
popularity.sort_values('sales_value', ascending=False, inplace=True)
popularity['sales_log'] = np.log(popularity['sales_value'])
popularity['weight'] = popularity['sales_log'] / popularity['sales_log'].sum()
print(f'Сумма weight = {popularity.weight.sum()}')

Сумма weight = 1.0


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [8]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """

    items = np.array(items_weights.item_id)
    weight = np.array(items_weights.weight)
    recs = np.random.choice(items, size=n, replace=False, p=weight)
    
    return recs.tolist()

In [9]:
%%time

items = data_train.item_id.unique()

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(popularity[['item_id', 'weight']], n=5))

CPU times: user 8.5 s, sys: 335 ms, total: 8.83 s
Wall time: 11.1 s


In [10]:
result.head(2)

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9296564, 15566862, 15971767, 7115533, 984861]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1042966, 924990, 968826, 1050516, 992765]"


### Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

### 2.1 Random recommendation и Weighted Random Recommendation

In [11]:
%%time

top = 5000

items = popularity.item_id.unique()[:top]
popularity = popularity[:top]
popularity['weight'] = popularity['sales_log'] / popularity['sales_log'].sum()

CPU times: user 11.2 ms, sys: 0 ns, total: 11.2 ms
Wall time: 12.5 ms


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [13]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(popularity[['item_id', 'weight']], n=5))

result.head(2)

CPU times: user 9.6 s, sys: 141 ms, total: 9.74 s
Wall time: 13.8 s


In [14]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1055863, 13841744, 952408, 1116020, 861494]","[12427727, 1120749, 1978737, 12947252, 9935348]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[968215, 874061, 964594, 1092026, 1105623]","[1163337, 1008140, 15683257, 2312974, 13213032]"


### 2.2 Item-Item Recommender / ItemKNN

In [15]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [16]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [17]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 
user_item_matrix = user_item_matrix.astype(float) 


sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [19]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [20]:
model = ItemItemRecommender(K=3, num_threads=4) 

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

In [21]:
result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)])

### 2.3 Косинусное сходство и CosineRecommender

In [22]:
%%time

model = CosineRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 2.64 s, sys: 21.4 ms, total: 2.66 s
Wall time: 2.12 s


In [23]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)])

CPU times: user 72.7 ms, sys: 939 µs, total: 73.7 ms
Wall time: 74.2 ms


In [24]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1055863, 13841744, 952408, 1116020, 861494]","[12427727, 1120749, 1978737, 12947252, 9935348]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 1127831, 981760, 961554, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[968215, 874061, 964594, 1092026, 1105623]","[1163337, 1008140, 15683257, 2312974, 13213032]","[1082185, 981760, 1098066, 6534178, 826249]","[1082185, 1098066, 826249, 883404, 981760]"


### 2.4 TF-IDF взвешивание и TFIDFRecommender

In [25]:
%%time

model = TFIDFRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 2.69 s, sys: 34 ms, total: 2.72 s
Wall time: 2.17 s


In [26]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)])

CPU times: user 67.2 ms, sys: 3.95 ms, total: 71.2 ms
Wall time: 75 ms


### 2.5 "Трюк"

In [57]:
%%time

model = ItemItemRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 3.07 s, sys: 35.7 ms, total: 3.1 s
Wall time: 3.18 s


In [58]:
%%time

result['own_purchases_k3'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[999999], 
                                    recalculate_user=False)])

CPU times: user 95.9 ms, sys: 0 ns, total: 95.9 ms
Wall time: 113 ms


In [63]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,itemitem,cosine,tfidf,own_purchases_k1,own_purchases_k5,own_purchases_k3
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1055863, 13841744, 952408, 1116020, 861494]","[12427727, 1120749, 1978737, 12947252, 9935348]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 1127831, 981760, 961554, 1098066]","[1082185, 981760, 1127831, 961554, 840361]","[999999, 1082185, 1029743, 995785, 1004906]","[999999, 1082185, 1029743, 995785, 1004906]","[999999, 1082185, 981760, 995242, 1029743]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[968215, 874061, 964594, 1092026, 1105623]","[1163337, 1008140, 15683257, 2312974, 13213032]","[1082185, 981760, 1098066, 6534178, 826249]","[1082185, 1098066, 826249, 883404, 981760]","[1082185, 1098066, 826249, 883404, 981760]","[999999, 1082185, 1098066, 6534178, 1127831]","[999999, 1082185, 1098066, 6534178, 1127831]","[999999, 1082185, 981760, 1098066, 6534178]"


### 3 Измерим качество по precision@5

In [35]:
#from google.colab import files

#files.upload()

In [41]:
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k

In [62]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0072:weighted_random_recommendation
0.0004:random_recommendation
0.219:itemitem
0.1639:cosine
0.1688:tfidf
0.1797:own_purchases_k1
0.1797:own_purchases_k5
0.1861:own_purchases_k3
